In [1]:
# importing Fastai vision and all the required matrices
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.vision.data import ImageDataBunch
from fastai import *
import numpy as np
import pandas as pd
import scipy.io as sio
import warnings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('agg')
%matplotlib inline
warnings.filterwarnings("ignore")
import sys

In [2]:
# Loading the dataset and Data Augmentation for training and validation data to avoid overfitting

data = ImageDataBunch.from_folder('pill_dataset','train','valid',ds_tfms=get_transforms(do_flip=False, flip_vert=True, max_rotate=5.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75),size=224,bs=16).normalize(imagenet_stats)

In [3]:
# Plot the sample batch
data.show_batch()
plt.savefig('sample_of_batch_images.png')

In [5]:
# class names and number of classes
print(data.classes)
l = len(data.classes)
print(l)

['3B-Medi', 'Agifamcin-300', 'Agifovir-E', 'Alpha-Choay', 'Alprazolam-Mylan-0.5mg', 'Ambron', 'Ameflu-Daytime', 'Amlodipin', 'Apha-Bevagyl', 'Arcalion-200', 'Asanclar', 'Atorvastatin', 'Azitnic', 'BNGreen', 'BP', 'Bepoz-5', 'Bermoric', 'Betaloc-ZOK', 'Bineurox', 'Biolac-A++', 'Biotin', 'Biseptol', 'Blackmores-Celery-3000', 'BrainIQ', 'Bạch-Địa-Căn', 'Cactimin-C500', 'Cenovis', 'Cephalexin', 'Clarityne', 'Colchicin', 'Cosele', 'Deflucold-Day', 'Dexipharm-15', 'Dopharalgic', 'Dorithricin', 'Efferalgan', 'Emb-Fatol', 'Feburic', 'Fexophar', 'Fugacar', 'Gabarica-400', 'Glotadol-F', 'Glotadol-Power', 'Hepromax', 'Homiginmin-Ginseng', 'Itraconazole', 'Lacbiopro-Bidiphar', 'Levofloxacin', 'Livetin', 'Livolin-H', 'Lopera', 'Ludox', 'Lục-Vị-F', 'Macdin', 'Maxrogyl', 'Maxxhair', 'Maxxmucous-AB-30', 'Medrol', 'Mobium', 'Moxilen', 'Myderison', 'Neurontin', 'Nonzoli', 'Nutrios', 'OpeAzitro', 'Opetrypsin-4200', 'Pacemin', 'Pantoprazol', 'Patrotadin', 'Power-Brain', 'Pyrazinamide', 'Regatonic', 'Rined

In [6]:
# For the training, using DenseNet201 and batch size of 16.
epochs = 10
learn = cnn_learner(data, models.densenet201, metrics=accuracy)
learn.fit_one_cycle(epochs)

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
# Loss plot for DenseNet201
learn.recorder.plot_losses()
plt.savefig('DenseNet201 Loss.png')


In [ ]:
# Accuracy plot for DenseNet201
learn.recorder.plot_metrics()
plt.savefig('DenseNet201 Accuracy.png')

In [16]:
# Plotting the loss and accuracy for DenseNet201
data = learn.recorder.plot_losses()
data_acc = learn.recorder.plot_metrics()

steps_per_epoch = int(len(data[1])/epochs)
# print(steps_per_epoch)
training_loss = []
for st in range(epochs+1):
    if st == 0:
        training_loss.append(data[1][st])
    else:
        training_loss.append(data[1][(st*steps_per_epoch)-1])

total_epochs = [i for i in range(epochs+1)]
epochs_val = total_epochs.copy()
del epochs_val[0]
val_loss = data[-1]

figure, axis = plt.subplots(2, 2)
# For Train Loss
axis[0, 0].plot(total_epochs, training_loss, label='Train Loss')
axis[0, 0].set_title("Train Loss")
# For Test Loss
axis[0, 1].plot(epochs_val, val_loss, label='Test Loss')
axis[0, 1].set_title("Test Loss")
# For Test Loss
axis[1, 0].plot(epochs_val, data_acc, label='Test Accuracy')
axis[1,0].set_xlabel("Test Accuracy")
# For Overall Model Evaluation
axis[1, 1].plot(total_epochs, training_loss, label='Train Loss')
axis[1, 1].plot(epochs_val, val_loss, label='Test Loss')
axis[1, 1].plot(epochs_val, data_acc, label='Test Accuracy')
# axis[1, 1].set_title()
axis[1,1].set_xlabel("All Training Plots")
axis[1, 1].legend()

plt.savefig('training_plot DenseNet201.png')

In [17]:
# calculate the accuracy of the model
preds,y, loss = learn.get_preds(with_loss=True)
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc))

The accuracy is 1.0 %.


In [18]:
# Confusion matrix for DenseNet201
interp = ClassificationInterpretation.from_learner(learn)
fig1=interp.plot_confusion_matrix(figsize=(25,25),return_fig=True)
fig1.savefig('DenseNet201_cm.png')


In [ ]:
# Plotting most confused classes for DenseNet201
interp.plot_top_losses(10, figsize=(15,11))
plt.savefig('DenseNet201_most_confused.png')

In [ ]:
# plotting learning rate for DenseNet201
learn.lr_find()
learn.recorder.plot(sugession=True)
plt.savefig('DenseNet201 Learning Rate.png')
